In [1]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
data = pd.read_csv('./data.csv')
data.head()

,ADR,text,category
0,ADR-2022-100004022857,"# Lee, Terrilynn, Marie 000029128 I=0000 29128...",Award Letter(s)
1,ADR-2022-100004025366,BeBenefits A Service of the Department of Vete...,Award Letter(s)
2,ADR-2022-100004026667,Service Verification Letter DEPARTMENT OF VETE...,Award Letter(s)
3,ADR-2022-100004027737,AB8 Benefit Verification Letter DEPARTMENT OF ...,Award Letter(s)
4,ADR-2022-100004029343,The Prudential Insurance Company of America Pr...,Award Letter(s)


In [4]:
data.isnull().sum()

ADR           0
text        502
category      0
dtype: int64

In [5]:
data.dropna(inplace=True)
data.shape

(53865, 3)

In [6]:
data['category'].value_counts()

Purchase Agreement                     12676
Tax Return - Personal                  11233
Bank Statement                          8102
Retirement Account Statement(s)         6354
Hazard Insurance Dec Page - Initial     3698
Mortgage Statement                      3077
Rental Agreements(s)                    3008
Divorce Decree                          2383
Paystub                                 1655
K1s                                      846
W-2(s) / 1099(s)                         450
Award Letter(s)                          383
Name: category, dtype: int64

In [7]:
data['text'][10000]

'SCHEDULE C Profit or Loss From Business OMB No. 1545-0074 (Form 1040) (Sole Proprietorship) 2021 Go to www.irs.gov/ScheduleC for instructions and the latest information. Department of the Treasury Attachment Internal Revenue Service (99) Attach to Form 1040, 1040-SR, 1040-NR, or 1041; partnerships generally must file Form 1065. Sequence No. 09 Name of proprietor Social security number (SSN) JAMES D HARBIN 423-06-2557 A Principal business or profession, including product or service (see instructions) B Enter code from instructions TRUCK DRIVER C Business name. If no separate business name, leave blank. D Employer ID number (EIN) (see instr.) E Business address (including suite or room no.) > 838 FRY GAP RD City, town or post office, state, and ZIP code Arab, AL 35016 F Accounting method: (1) x Cash (2) Accrual (3) Other (specify)» Did you "materially participate" in the operation of this business during 2021? If "No," see instructions for limit on losses X Yes No G H If you started or 

In [8]:
data['category'][10000]

'Tax Return - Personal'

In [9]:
temp = []
cats = ['Mortgage Statement']
for category in data['category']:
    if category not in cats:
        temp.append('others')
    else:
        temp.append(category)
data['new_category'] = temp
data.head()

,ADR,text,category,new_category
0,ADR-2022-100004022857,"# Lee, Terrilynn, Marie 000029128 I=0000 29128...",Award Letter(s),others
1,ADR-2022-100004025366,BeBenefits A Service of the Department of Vete...,Award Letter(s),others
2,ADR-2022-100004026667,Service Verification Letter DEPARTMENT OF VETE...,Award Letter(s),others
3,ADR-2022-100004027737,AB8 Benefit Verification Letter DEPARTMENT OF ...,Award Letter(s),others
4,ADR-2022-100004029343,The Prudential Insurance Company of America Pr...,Award Letter(s),others


In [10]:
data['new_category'].value_counts()

others                50788
Mortgage Statement     3077
Name: new_category, dtype: int64

In [11]:
df = data.sample(frac=0.7, random_state=10)
df.shape

(37706, 4)

In [12]:
test = data.drop(df.index).reset_index(drop=True)
test.shape

(16159, 4)

In [13]:
test['new_category'].value_counts()

others                15280
Mortgage Statement      879
Name: new_category, dtype: int64

In [14]:
test.head()

,ADR,text,category,new_category
0,ADR-2022-100004026667,Service Verification Letter DEPARTMENT OF VETE...,Award Letter(s),others
1,ADR-2022-100004027737,AB8 Benefit Verification Letter DEPARTMENT OF ...,Award Letter(s),others
2,ADR-2022-100004029343,The Prudential Insurance Company of America Pr...,Award Letter(s),others
3,ADR-2022-200004024398,online. UNITED STATES UNITED STATES OFFICE OF ...,Award Letter(s),others
4,ADR-2022-100004026927,Benefit Summary Letter DEPARTMENT OF VETERANS ...,Award Letter(s),others


In [15]:
df.reset_index(drop=True, inplace=True)
df.head()

,ADR,text,category,new_category
0,ADR-2022-200004312487,x LINE OF CREDIT Account Number: 0016100107381...,Mortgage Statement,Mortgage Statement
1,ADR-2022-200000369557,"REAL MANAGEMENT .. 1751 Samo Road, Suite 5, Me...",Rental Agreements(s),others
2,ADR-2022-100004069788,Form 1040-SR (2020) DENNIS M & LINDA DUGAN 227...,Tax Return - Personal,others
3,ADR-2022-200000369415,"e) In the event of reconciliation, the Parties...",Divorce Decree,others
4,ADR-2022-100004024411,"4/20/22, 8:58 AM Fidelity Netbenefits - SOD de...",Retirement Account Statement(s),others


In [16]:
possible_labels = df.new_category.unique()

label_dict = {}
label_dict['others']=0
for i,cat in enumerate(cats):
    label_dict[cat]=i+1
#for index, possible_label in enumerate(possible_labels):
#    label_dict[possible_label] = index
label_dict

{'others': 0, 'Mortgage Statement': 1}

In [17]:
df['label'] = df.new_category.replace(label_dict)
df.head()

,ADR,text,category,new_category,label
0,ADR-2022-200004312487,x LINE OF CREDIT Account Number: 0016100107381...,Mortgage Statement,Mortgage Statement,1
1,ADR-2022-200000369557,"REAL MANAGEMENT .. 1751 Samo Road, Suite 5, Me...",Rental Agreements(s),others,0
2,ADR-2022-100004069788,Form 1040-SR (2020) DENNIS M & LINDA DUGAN 227...,Tax Return - Personal,others,0
3,ADR-2022-200000369415,"e) In the event of reconciliation, the Parties...",Divorce Decree,others,0
4,ADR-2022-100004024411,"4/20/22, 8:58 AM Fidelity Netbenefits - SOD de...",Retirement Account Statement(s),others,0


In [18]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [19]:
df['data_type'] = ['not_set']*df.shape[0]
df.head()

,ADR,text,category,new_category,label,data_type
0,ADR-2022-200004312487,x LINE OF CREDIT Account Number: 0016100107381...,Mortgage Statement,Mortgage Statement,1,not_set
1,ADR-2022-200000369557,"REAL MANAGEMENT .. 1751 Samo Road, Suite 5, Me...",Rental Agreements(s),others,0,not_set
2,ADR-2022-100004069788,Form 1040-SR (2020) DENNIS M & LINDA DUGAN 227...,Tax Return - Personal,others,0,not_set
3,ADR-2022-200000369415,"e) In the event of reconciliation, the Parties...",Divorce Decree,others,0,not_set
4,ADR-2022-100004024411,"4/20/22, 8:58 AM Fidelity Netbenefits - SOD de...",Retirement Account Statement(s),others,0,not_set


In [20]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [21]:
df.groupby(['new_category', 'label', 'data_type']).count()

ADR   text  category
new_category       label data_type                        
Mortgage Statement 1     train       1758   1758      1758
                         val          440    440       440
others             0     train      28406  28406     28406
                         val         7102   7102      7102

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [23]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    truncation=True,
    return_tensors='pt'
)

/data/mmortgage/MultiPage_Classification/classification_service/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [138]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [139]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [140]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [141]:
batch_size=4
dataloader_train = DataLoader(dataset_train, 
                              shuffle=True,
                              num_workers=0, 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   shuffle=False,
                                   num_workers=0, 
                                   batch_size=batch_size)

In [142]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8,
                  no_deprecation_warning=True)
                  
epochs = 2

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [143]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [144]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs.loss
        logits = outputs.logits
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [145]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
      
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
                }       

        outputs = model(**inputs)
        loss = outputs.loss
        loss_train_total += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(), f'./model/model_Purchase_Rental_Aggrement/finetuned_BERT_epoch_{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total/len(dataloader_train)
    
    tqdm.write(f'Training loss: {loss_train_avg}')
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/8013 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.060446900530425306
Validation loss: 0.030480083375175115
F1 Score (Weighted): 0.9943308464091146


Epoch 2:   0%|          | 0/8013 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.015122957348165676
Validation loss: 0.019398684074371533
F1 Score (Weighted): 0.9968132054804874


### Evaluation

In [146]:
test['label'] = test.new_category.replace(label_dict)
test.head()

,ADR,text,category,new_category,label
0,ADR-2022-100004026667,Service Verification Letter DEPARTMENT OF VETE...,Award Letter(s),others,0
1,ADR-2022-100004027737,AB8 Benefit Verification Letter DEPARTMENT OF ...,Award Letter(s),others,0
2,ADR-2022-100004029343,The Prudential Insurance Company of America Pr...,Award Letter(s),others,0
3,ADR-2022-200004024398,online. UNITED STATES UNITED STATES OFFICE OF ...,Award Letter(s),others,0
4,ADR-2022-100004026927,Benefit Summary Letter DEPARTMENT OF VETERANS ...,Award Letter(s),others,0


In [147]:
encoded_data_test = tokenizer.batch_encode_plus(
    test.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=512, 
    truncation=True,
    return_tensors='pt'
)

In [148]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test.label.values)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [149]:
batch_size=16
dataloader_test = DataLoader(dataset_test, 
                              shuffle=True,
                              num_workers=0, 
                              batch_size=batch_size)

In [150]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)

model.load_state_dict(torch.load('./model/model_Purchase_Rental_Aggrement/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))
model.eval()
loss, predictions, true_values = evaluate(dataloader_test)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [151]:
accuracy_per_class(predictions, true_values)

Class: others
Accuracy: 11385/11399

Class: Purchase Agreement
Accuracy: 3793/3837

Class: Rental Agreements(s)
Accuracy: 884/923



In [168]:
test['category'][7063]

'Hazard Insurance Dec Page - Initial'

In [169]:
#text = test['text'][3000]
encoded_data_text = tokenizer.encode_plus(
    text, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    truncation=True,
    return_tensors='pt'
)

In [170]:
input_ids_text = encoded_data_text['input_ids']
attention_masks_text = encoded_data_text['attention_mask']
labels_text = encoded_data_text['token_type_ids']

In [171]:
label_dict_inv=dict()
for label in label_dict:
    label_dict_inv[label_dict[label]]=label
label_dict_inv 

{0: 'others', 1: 'Purchase Agreement', 2: 'Rental Agreements(s)'}

In [172]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.load_state_dict(torch.load('./model/model_Purchase_Rental_Aggrement/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))
model.eval()
outputs = model(input_ids_text, attention_masks_text, labels_text)
outputs

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

SequenceClassifierOutput(loss=None, logits=tensor([[ 7.9198, -3.6594, -3.7391]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [173]:
label_dict_inv[np.argmax(outputs.logits.detach().numpy())]

'others'